In [1]:
%cd /content/drive/MyDrive/研究室/研究_M2/src

/content/drive/MyDrive/研究室/研究_M2/src


In [2]:
import datetime
from pytz import timezone

def Pull_Datetime():
    dates=datetime.datetime.now(timezone('Asia/Tokyo'))
    exept_microsec=dates.strftime("%Y-%m-%d-%H-%M")
    print('[Today : %s]' % str(exept_microsec))

    DATETIME=exept_microsec
    return DATETIME

DATETIME=Pull_Datetime()

[Today : 2021-08-24-00-36]


In [3]:
KEYWORDS_FileName= '2017'

In [4]:
import os
OUTPUT_PATH =  './Assets/Assets_Output/'
OUTPUT_DIR=OUTPUT_PATH+ "PersonalEstim_%s/" % DATETIME
OUTPUT_DIR_RESULTS=OUTPUT_PATH+ "PersonalEstim_%s/results/" % DATETIME

KEYWORDS_TypeMBTI = '2017-2018'

try:
  os.mkdir(OUTPUT_DIR)
except:
  print(OUTPUT_DIR, '    already exists')
  print()
try:
  os.mkdir(OUTPUT_DIR_RESULTS)
except:
  print(OUTPUT_DIR_RESULTS, '    the directory already exists')
  print()

In [5]:
import yaml
import os # For saving access tokens and for file management when creating and adding to the dataset
import requests # For sending GET requests from the API
import Twitter_API_Key

def Create_url_SearchTweets(keyword, start_date, end_date, max_results = 10):
    #Change to the endpoint you want to collect data from
    search_url = "https://api.twitter.com/2/tweets/search/all" 
    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
                    
    return (search_url, query_params)

In [6]:
def Connect_to_endpoint(url, params, next_token = None):
    #prepare params
    params['next_token'] = next_token   #params object received from create_url function
    #pull bearer token
    bearer_token = Twitter_API_Key.Auth_FullAchieve()
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    #request Twtitter
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
        
    return response.json()

In [15]:
import dateutil.parser
import pandas as pd

def Search_Tweets(keyword, start_time, end_time, max_results, next_token, KEYWORDS_FileName, count_index):
  url = Create_url_SearchTweets(keyword, start_time, end_time, max_results)
  print(url)
  json_SearchTweets= Connect_to_endpoint(url[0], url[1], next_token)
  # json_SearchTweets
  print(json_SearchTweets)
  list_EachData=json_SearchTweets['data']

  list_csv=[]
  list_csv.clear()
  list_header=['author_id', 'text', 'tweet_id', 'tweet_time', 'lang']
  for element in list_EachData:
    #print(element['text'])
    tweet_AuthorID=element['author_id']
    tweet_Texts=element['text']
    tweet_id = element['id']
    tweet_time = dateutil.parser.parse(element['created_at'])
    tweet_lang = element['lang']

    list_sub_row=[tweet_AuthorID, tweet_Texts, tweet_id, tweet_time, tweet_lang]
    list_csv.append(list_sub_row)

  df_SearchTweets=pd.DataFrame(list_csv, columns=list_header)
  df_SearchTweets.to_csv('%sUserList_%s.csv' % (OUTPUT_DIR_RESULTS, count_index))
  print('%sUserList_%s.csvを作成しました。'% (OUTPUT_DIR_RESULTS, count_index))

  return df_SearchTweets, tweet_time

In [9]:
import time

#Inputs for the request
keyword = "#16Personalities 16personalities.com lang:ja"
max_results = 500
start_time = "2017-01-01T00:00:00.000Z"
end_time='2017-12-31T23:59:59.000Z'
# start_time = "2018-12-31T00:00:00.000Z"
# end_time='2018-12-31T23:59:59.000Z'
df_2017to2018=None

for count_index in range(0, 10000):
  print(count_index)
  next_token = None
  df_SearchTweets, end_time=Search_Tweets(keyword, start_time, end_time, max_results, next_token, KEYWORDS_TypeMBTI, count_index)

  b=str(end_time).replace(' ', 'T')
  #print(b.split('+'))
  c=b.split('+')
  end_time=c[0] + '.000Z'
  #print(d)
  
  df_2017to2018=pd.concat([df_2017to2018, df_SearchTweets], axis=0)
  # except:
  #   print('passしました')
  #   pass

  #time.sleep(5)
  print(end_time)
  
df_2017to2018.to_csv('%sUserListFinal_%s.csv' % (OUTPUT_DIR_RESULTS, KEYWORDS_FileName))
#df_SearchTweets.to_csv('%sUserList_%s.csv' % (OUTPUT_DIR_RESULTS, KEYWORDS_FileName))

0
('https://api.twitter.com/2/tweets/search/all', {'query': '#16Personalities 16personalities.com lang:ja', 'start_time': '2017-01-01T00:00:00.000Z', 'end_time': '2017-12-31T23:59:59.000Z', 'max_results': 500, 'expansions': 'author_id,in_reply_to_user_id,geo.place_id', 'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source', 'user.fields': 'id,name,username,created_at,description,public_metrics,verified', 'place.fields': 'full_name,id,country,country_code,geo,name,place_type', 'next_token': {}})
Endpoint Response Code: 200
./Assets/Assets_Output/PersonalEstim_2021-08-24-00-36/results/UserList_0.csvを作成しました。
2017-12-19T16:28:07.000Z
1
('https://api.twitter.com/2/tweets/search/all', {'query': '#16Personalities 16personalities.com lang:ja', 'start_time': '2017-01-01T00:00:00.000Z', 'end_time': '2017-12-19T16:28:07.000Z', 'max_results': 500, 'expansions': 'author_id,in_reply_to_user_id,geo.place_id', 

KeyError: ignored

In [11]:
df_SearchTweets

,author_id,text,tweet_id,tweet_time,lang
0,758875042994204673,RT @nanaskyyy: 私の性格タイプは “起業家” です。あなたのタイプは何ですか？...,817974127684296707,2017-01-08 06:00:03+00:00,ja
1,2499059894,私の性格タイプは “擁護者” です。あなたのタイプは何ですか？ https://t.co/s...,817974031164928000,2017-01-08 05:59:40+00:00,ja
2,352949960,私の性格タイプは “起業家” です。あなたのタイプは何ですか？ https://t.co/5...,817973347581497345,2017-01-08 05:56:57+00:00,ja
3,2956855579,私の性格タイプは “冒険家” です。あなたのタイプは何ですか？ https://t.co/o...,817969979374989312,2017-01-08 05:43:34+00:00,ja
4,622561432,私の性格タイプは “提唱者” です。あなたのタイプは何ですか？ https://t.co/q...,817961975195963394,2017-01-08 05:11:46+00:00,ja
...,...,...,...,...,...
462,879292549,私の性格タイプは “論理学者” です。あなたのタイプは何ですか？ https://t.co/...,815531829243453440,2017-01-01 12:15:14+00:00,ja
463,781783504199614464,私の性格タイプは “建築家” です。あなたのタイプは何ですか？ https://t.co/O...,815523615915446272,2017-01-01 11:42:36+00:00,ja
464,589077235,私の性格タイプは “エンターテイナー” です。あなたのタイプは何ですか？ https://t...,815469649064128512,2017-01-01 08:08:09+00:00,ja
465,1017366481,私の性格タイプは “擁護者” です。あなたのタイプは何ですか？ https://t.co/S...,815456491708633088,2017-01-01 07:15:52+00:00,ja


In [12]:
df_2017to2018

,author_id,text,tweet_id,tweet_time,lang
0,1620241032,私の性格タイプは “建築家” です。あなたのタイプは何ですか？ https://t.co/c...,947493403071098881,2017-12-31 15:43:25+00:00,ja
1,708171936782622720,私の性格タイプは “仲介者” です。あなたのタイプは何ですか？ https://t.co/y...,947481939396608001,2017-12-31 14:57:52+00:00,ja
2,4256765894,私の性格タイプは “広報運動家” です。あなたのタイプは何ですか？ https://t.co...,947480443783299073,2017-12-31 14:51:55+00:00,ja
3,711443626774016000,仲介者型の性格 (INFP-A / INFP-T) https://t.co/HtTGyon...,947466597161037824,2017-12-31 13:56:54+00:00,ja
4,711443626774016000,私の性格タイプは “仲介者” です。あなたのタイプは何ですか？ https://t.co/t...,947462646881595395,2017-12-31 13:41:12+00:00,ja
...,...,...,...,...,...
462,879292549,私の性格タイプは “論理学者” です。あなたのタイプは何ですか？ https://t.co/...,815531829243453440,2017-01-01 12:15:14+00:00,ja
463,781783504199614464,私の性格タイプは “建築家” です。あなたのタイプは何ですか？ https://t.co/O...,815523615915446272,2017-01-01 11:42:36+00:00,ja
464,589077235,私の性格タイプは “エンターテイナー” です。あなたのタイプは何ですか？ https://t...,815469649064128512,2017-01-01 08:08:09+00:00,ja
465,1017366481,私の性格タイプは “擁護者” です。あなたのタイプは何ですか？ https://t.co/S...,815456491708633088,2017-01-01 07:15:52+00:00,ja


In [13]:
df_2017to2018.to_csv('%sUserListFinal_%s.csv' % (OUTPUT_DIR_RESULTS, KEYWORDS_FileName))

In [17]:
print('%sUserListFinal_%s.csv' % (OUTPUT_DIR_RESULTS, KEYWORDS_FileName))

./Assets/Assets_Output/PersonalEstim_2021-08-24-00-36/results/UserListFinal_2017.csv


In [18]:
list_AuthorID=df_2017to2018['author_id'].values.tolist()

In [20]:
list_AuthorID.sort()

In [21]:
print(list_AuthorID)

['1000405860', '1000793498', '1000856706', '100106333', '1001153851', '1001153851', '1001200189', '1001388404', '1001454445', '1001497244', '100159745', '1001657288', '1001661216', '100169552', '1001777072', '100181366', '100181366', '100216102', '100220967', '100221811', '100224813', '100240066', '100241675', '100246438', '100275798', '100276194', '100284659', '1003112088', '1003302416', '100331880', '1003534698', '100366913', '100385374', '1003948699', '1004103169', '100415776', '100424432', '1004255096', '1004302928', '1004311368', '100439954', '1004421128', '100442673', '100491853', '100493243', '100505173', '1005375080', '1005425216', '100546552', '100551049', '1005586064', '1005658610', '1005879901', '100595489', '100615589', '1006461944', '100665135', '100666656', '1006688292', '10071892', '100733638', '100743653', '100754762', '100759092', '1007898948', '100795885', '1008295765', '1008405062', '1008485912', '1008592706', '100871270', '1008801582', '1008859530', '100888220', '10

In [27]:
with open('/content/drive/MyDrive/sample.txt', 'w') as f:
  for i in range(len(list_AuthorID)):
    print(list_AuthorID[i], file=f)


with open('/content/drive/MyDrive/sample.txt') as f:
  list_AuthorID=f.readlines()
  for i in range(len(list_AuthorID)):
    list_AuthorID[i]=list_AuthorID[i].rstrip()
  print(list_AuthorID)

['1000405860', '1000793498', '1000856706', '100106333', '1001153851', '1001153851', '1001200189', '1001388404', '1001454445', '1001497244', '100159745', '1001657288', '1001661216', '100169552', '1001777072', '100181366', '100181366', '100216102', '100220967', '100221811', '100224813', '100240066', '100241675', '100246438', '100275798', '100276194', '100284659', '1003112088', '1003302416', '100331880', '1003534698', '100366913', '100385374', '1003948699', '1004103169', '100415776', '100424432', '1004255096', '1004302928', '1004311368', '100439954', '1004421128', '100442673', '100491853', '100493243', '100505173', '1005375080', '1005425216', '100546552', '100551049', '1005586064', '1005658610', '1005879901', '100595489', '100615589', '1006461944', '100665135', '100666656', '1006688292', '10071892', '100733638', '100743653', '100754762', '100759092', '1007898948', '100795885', '1008295765', '1008405062', '1008485912', '1008592706', '100871270', '1008801582', '1008859530', '100888220', '10